# Synthetic generation of Heston data

Here, we generate labeled Heston data for various tasks and figures in the paper.

In [1]:
# Standard library imports
import os
import datetime
from os.path import dirname as up
import numpy as np
import pandas as pd
import QuantLib as ql

from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_lets_be_rational.exceptions import BelowIntrinsicException
import sklearn.utils
import logging

# Important directories
deep_cal_dir = up(up(os.getcwd()))
code_dir = up(os.getcwd())

# Logging stuff
logger = logging.getLogger("heston")
logger.setLevel(logging.INFO)
fh = logging.FileHandler(code_dir + "/logs/heston_simulation.log")    
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

In [2]:
def heston_pricer(lambd, vbar, eta, rho, v0, r, q, tau, S0, K):
    """
    Computes European Call price under Heston dynamics with closedform solution.

    :param lambd: mean-reversion speed
    :param vbar: long-term average variance
    :param eta: volatility of variance
    :param rho: correlation between stock and vol
    :param v0: spot variance
    :param r: risk-free interest rate
    :param q: dividend rate
    :param tau: time to maturity in years (year = 365 days)
    :param S0: initial spot price
    :param K: strike price

    :return: Heston price, Black-Scholes implied volatility
    :rtype: float, float

    """
    today = datetime.date.today()
    calculation_date = ql.Date(today.day, today.month, today.year)
     # Which convention should I use here?
    day_count = ql.Actual365Fixed()
    ql.Settings.instance().evaluationDate = calculation_date

    # Option data
    option_type = ql.Option.Call
    payoff = ql.PlainVanillaPayoff(option_type, K)
    # 'Dirty' way: Convert year fraction to days and add to today
    maturity_date = calculation_date + int(round(tau * 365))
    exercise = ql.EuropeanExercise(maturity_date)
    european_option = ql.VanillaOption(payoff, exercise)

    # Heston process  
    spot_handle = ql.QuoteHandle(
        ql.SimpleQuote(S0))

    flat_ts = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, r, day_count))

    dividend_yield = ql.YieldTermStructureHandle(
        ql.FlatForward(calculation_date, q, day_count))

    heston_process = ql.HestonProcess(flat_ts,
                                      dividend_yield,
                                      spot_handle,
                                      v0,
                                      lambd,
                                      vbar,
                                      eta,
                                      rho)

    engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process), 1E-15, 1000000)
    european_option.setPricingEngine(engine)

    # Check for numerical instabilities
    try:
        
        price = european_option.NPV()
        
        if price <= 0 or price < (S0 - K):
        
            iv = np.nan

            logger.debug('NumStabProblem: Price {}. Intrinsic {}. Time {}. Strike {}.'.format(
                        price, S0-K, tau, K))

            return price, iv
    
        else:

            logger.debug('Success: Price {} > intrinsic {}.'.format(price, S0-K))

            iv = implied_volatility(price, S0, K, tau, r, 'c')

            return price, iv
        
    except RuntimeError:
        
        logger.info('RuntimeError: Intrinsic {}. Time {}. Strike {}.'.format(S0-K, tau, K))
        
        price = np.nan
        iv = np.nan
        
        return price, iv

Sanitycheck against Premia Pricer

In [ ]:
# Market parameters
S0 = 1
r = 0 

# Heston parameters 
lambd = 1.3253 
vbar = 0.0354 
eta = 0.3877 
rho = -0.7165 
v0 = 0.0174 
tau = 0.3 
K = 0.9
q = 0

heston_pricer(lambd, vbar, eta, rho, v0, r, q, tau, S0, K)

## Smile data generation

In [ ]:
nb_samples = 1000

# Market parameters
S0 = 1
r = 0
q = 0

# Heston parameters
lambd = 1.3253
vbar = 0.0354
eta = 0.3877
rho = -0.7165
v0 = 0.0174
T = 0.05

log_moneyness = np.linspace(-0.1, 0.28, nb_samples)
strikes = np.exp(log_moneyness)*S0

# Initialisation of df for labeled data
columns = ['strikes','log_moneyness', 'iv']
df = pd.DataFrame(np.zeros((nb_samples,3)), columns=columns)

# Filling in parameter data
df['strikes'] = strikes
df['log_moneyness'] = log_moneyness

In [ ]:
for i in range(nb_samples):
    
    K = df.strikes[i]
    
    # Calculate Black-Scholes implied vol from Heston price.
    price, iv = heston_pricer(lambd, vbar, eta, rho, v0, r, q, T, S0, K)
    
    df.loc[i, 'price'] = price
    df.loc[i, 'iv'] = iv   
    
df.dropna(inplace=True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
ax = df.plot(x='log_moneyness', y='iv')
fig = plt.gca()
plt.savefig('test.pdf')

In [ ]:
df.to_csv(deep_cal_dir +'/data/heston/volsmile_slice_0.3.csv', index=False)

## Labeled Data for Neural Network

Preparations 

In [3]:
# PARAMETERS
random_seed = 0
nb_samples = 10**6

# Heston parameter, bounds by Moodley (2005)
lambd_bounds = [0, 10]
vbar_bounds = [0, 1]
eta_bounds = [0, 5]
rho_bounds = [-1, 0]
v0_bounds = [0, 1]

# Market params
S0 = 1
r = 0
q = 0

# Import and preprocessing of import data
df0 = pd.read_csv(deep_cal_dir + '/data/raw_data/liquidity_bid_ask_spread/strike_maturity.csv')
df0.drop(['Unnamed: 0'], axis=1, inplace=True)

# Initialisation of df for labeled data
columns = ['lambda', 'vbar', 'eta', 'rho', 'v0', 'iv']
df = pd.DataFrame(np.zeros((nb_samples,6)), columns=columns)

# Merge df0 and df
df = pd.concat([df,df0], axis=1)

# Counters
count = 0
error_count = 0

#PRNG setting
np.random.seed(random_seed)

Now on to simulation of labeled data.

In [4]:
while count < nb_samples:
    
    # Take respective (strike, maturity) pair from precomputed data.
    K = df.strike[count]
    T = df.maturity[count]

    # Sample uniformly from Heston parameter space.
    lambd = np.random.uniform(lambd_bounds[0], lambd_bounds[1])
    vbar = np.random.uniform(vbar_bounds[0], vbar_bounds[1])
    eta = np.random.uniform(eta_bounds[0], eta_bounds[1])
    rho = np.random.uniform(rho_bounds[0], rho_bounds[1])
    v0 = np.random.uniform(v0_bounds[0], v0_bounds[1])

    # Calculate Black-Scholes implied vol from Heston price.
    price, iv = heston_pricer(lambd, vbar, eta, rho, v0, r, q, T, S0, K)
        
    df.loc[count, ['lambda', 'vbar', 'eta', 'rho', 'v0', 'iv']] = [lambd, vbar, eta, rho, v0, iv]

    count += 1     

    logger.info('Samples: {}'.format(count))

df.shape

(1000000, 8)

Shuffle output data, split into training/validation/test sets and write to .csv files.

In [5]:
# Delete all samples with NaN IV
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.truncate(after=990000)

df = sklearn.utils.shuffle(df)

# Dissecting labeled pairs into training, validation and testing sets.

train_df, validation_df, test_df, _ = np.split(df, [9*10**5, 9*10**5+45000, 9*10**5+90000], axis=0)

print('Shapes: \n train {}, validation {}, test {}'.format(train_df.shape, validation_df.shape,
                                                  test_df.shape))

Shapes: 
 train (900000, 8), validation (45000, 8), test (45000, 8)


In [7]:
# Write labeled data to .csv file.
train_df.to_csv(deep_cal_dir +'/data/heston/training_data1.csv', index=False)
validation_df.to_csv(deep_cal_dir + '/data/heston/validation_data1.csv', index=False)
test_df.to_csv(deep_cal_dir + '/data/heston/testing_data1.csv', index=False)

## Surface generation

In [ ]:
%matplotlib inline

nb_samples_moneyness = 200
nb_samples_maturities = 200

# Market parameters
S0 = 1
r = 0

# Heston parameters
lambd = 3.3
vbar = 0.96
eta = 2.26
rho = -.52
v0 = 0.53
maturities = np.linspace(1/365, 0.4, nb_samples_maturities)
moneyness = np.linspace(0.5, 1.5, nb_samples_moneyness)

# Initialisation of df for labeled data
columns = ['strikes','moneyness', 'maturity','iv']
df = pd.DataFrame(np.zeros((nb_samples_moneyness*nb_samples_maturities,4)), columns=columns)

# Filling df with values
xx, yy = np.meshgrid(maturities, moneyness)
df.maturity = xx.flatten()
df.moneyness = yy.flatten()
df['log_moneyness'] = np.log(df.moneyness)
df.strikes = S0/df.moneyness

In [ ]:
df.head(5)

In [ ]:
for i in range(len(df.index)):
    
    K = df.strikes[i]
    T = df.maturity[i]
    
    # Calculate Black-Scholes implied vol from Heston price.
    price, iv = heston_pricer(lambd, vbar, eta, rho, v0, r, T, S0, K)
    
    # Zero IV stems from numerical instability, so disregard
    if iv == 0:
        
        iv = np.nan
    
    df.loc[i, 'iv'] = iv   

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_trisurf(df.log_moneyness, df.maturity, df.iv, linewidth=0.2)

In [ ]:
df = df[df.maturity > 1/365]